# Summary
This notebook is used to automatically extract data from ATB yearly workbook file and generate summary accordingly to the specification.

## Getting Ready
- Here, we first import the package needed, and identify the path to files;
    - 2 paths to files are required here:
        1. raw_data_path: the path to the raw data file
        2. ancillary_path: the path to the ancillary file which declares the scope of tech and index of interest.

- Make 2 lists out of the ancillary file

In [130]:
import pandas as pd

In [131]:
raw_data_path = '/Users/zhixuan/PycharmProjects/ATB-Raw-Summarization/data/2022 v2 Annual Technology Baseline Workbook Corrected 7-21-2022.xlsx'

ancillary_path = '/Users/zhixuan/PycharmProjects/ATB-Raw-Summarization/data/ancillary.xlsx'

raw = pd.ExcelFile(raw_data_path)

In [132]:
ancillary = pd.read_excel(ancillary_path)

# make tech list out of ancillary
tech_list = list(ancillary['Tech'].values)

# make index list out of ancillary
index_list = list(ancillary['Index'].dropna().values)

# flush-flush
del ancillary

## Main loop
- Iterate over tech sheets needed

In [133]:
# get all the name of the sheets -> intersect with tech list
sheet_list = list(set(raw.sheet_names) & set(tech_list))
sub_tech_list = list(set(tech_list)-set(sheet_list))

In [134]:
# iterate through all sheets level techs
sheets = {}
for sheet in sheet_list:
    sheet_df = pd.read_excel(raw, sheet_name=str(sheet)).iloc[:, 9:].dropna(how='all')
    index = None
    head = []
    # iterate over rows
    for i in range(len(sheet_df)):
        # get the value on the current index column
        working_index = sheet_df.iloc[i, 0]

        # if it has null value
        if pd.isna(working_index):
            if not index is None:
                # assign the current index to the cell
                sheet_df.iloc[i, 0] = str(index)
        else:
            if working_index in index_list:
                index = working_index
                # keep the header row
                sheet_df.iloc[i-1, 0] = 'Header Row'
            else:
                sheet_df.iloc[i, 0] = None
                index = None

    sheet_df = sheet_df.dropna(how='any', subset=['Unnamed: 9'])
    sheets[str(sheet)] = sheet_df

In [135]:
sheets[sheet_list[0]]['Unnamed: 9'].unique()

array(['Header Row', 'Overnight Capital Cost ($/kW)',
       'Fixed Operation and Maintenance Expenses ($/kW-yr)',
       'Variable Operation and Maintenance Expenses ($/MWh)'],
      dtype=object)